<a href="https://colab.research.google.com/github/ustab/AORNEK_CALISMALAR-Veri_Bilimi/blob/main/EDAML_BASIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

sozluk = {'İsim':pd.Series(['Ada','Cem','Sibel','Ahmet','Mehmet','Ali','Veli',
          'Ayşe','Hüseyin','Necmi','Nalan','Namık']),
          'Meslek':pd.Series(['işçi','işçi','memur','serbest','serbest',None,None,
          'sigortacı','işsiz',None,None,'memur']),
          'Tarih':pd.Series(['11.11.2010','11.11.2010','11.11.2010','18.11.2011','18.11.2011',None,None,
          None,'11.11.2010',None,'18.11.2011','18.11.2011']),          
          'Yaş':pd.Series([21, 24, 25, 44, 31, 27, 35, 33, 42, 29, 41, 43]),
          'ÇocukSayısı':pd.Series([None, None, None, None, None, 1, 2, 0, None, None, None, None]),
          'Puan':pd.Series([89, 87, 77, 55, 70, 79, 73, 79, 54, 92, 61, 69])}
df = pd.DataFrame(sozluk)
df

In [ ]:
df.isnull().sum().sum()

In [ ]:
df.isnull().sum()

In [ ]:

#Eksik değer tablosu
def eksik_deger_tablosu(df): 
    eksik_deger = df.isnull().sum()
    eksik_deger_yuzde = 100 * df.isnull().sum()/len(df)
    eksik_deger_tablo = pd.concat([eksik_deger, eksik_deger_yuzde], axis=1)
    eksik_deger_tablo_son = eksik_deger_tablo.rename(
    columns = {0 : 'Eksik Değerler', 1 : '% Değeri'})
    return eksik_deger_tablo_son
  

In [ ]:
#%70 üzerinde null değer içeren kolonları sil
tr = len(df) * .3
df.dropna(thresh = tr, axis = 1, inplace = True)

df

In [ ]:
#Meslek özniteliğindeki Null değerleri 'Diğer' değeri ile doldur
df['Meslek'] = df['Meslek'].fillna('Diğer')

#Tarih özniteliğindeki Null değerleri Tarih benzersiz değerlerden ilki ile doldur
print(df['Tarih'].unique()[0])
df['Tarih'] = df['Tarih'].fillna(df['Tarih'].unique()[0])

df

In [ ]:

#Apply fonksiyonu kullanarak sınav başarı durumunu yeni öznitelik olarak ekle
def basari_durumu(puan):
    return (puan >= 70)

df['Geçti'] = df['Puan'].apply(basari_durumu)
df

In [ ]:

#Tarih özniteliğindeki yıl bilgisini kullanarak 'Yıl' isimli yeni bir öznitelik oluşturuyoruz
tarih = pd.to_datetime(df['Tarih'])
df['Yıl'] = tarih.dt.year

df

Bunu yapmanın en yaygın iki yolu Sklearn kütüphanesi altında yer alan LabelEncoder veya OneHotEncoder(ya da Pandas.get_dummies) sınıflarını kullanmaktır. Aslında bunları farklı iki yaklaşım olarak görmek mümkündür.

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() 
df['Geçti_Encoded']= label_encoder.fit_transform(df['Geçti'])

df
#view rawpython_sample_dataset_labelencoding1.py hosted with ❤ by GitHub

In [ ]:
df['Meslek_Encoded']= label_encoder.fit_transform(df['Meslek'])
df

In [ ]:
onehotencoder = preprocessing.OneHotEncoder()

In [ ]:
df['Meslek'] = pd.Categorical(df['Meslek'])
dfDummies = pd.get_dummies(df['Meslek'], prefix = 'Kat')
dfDummies

In [ ]:
df_new = pd.concat([df, dfDummies], axis=1)
df_new

In [ ]:
#Puan özniteliğini ölçeklendirmek istiyoruz
x = df[['Puan']].values.astype(float)

#Ölçeklendirme için MinMaxScaler fonksiyonunu kullanıyoruz.
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['Puan2'] = pd.DataFrame(x_scaled)

df

In [ ]:
import seaborn as sns
sns.boxplot(x=df['Puan'])

In [ ]:
Q1 = df.Puan.quantile(0.25)
Q2 = df.Puan.quantile(0.5)
Q3 = df.Puan.quantile(0.75)
Q4 = df.Puan.quantile(1)
IQR = Q3 - Q1

print("Q1-->", Q1)
print("Q3-->", Q3)
print("Q2-->", Q2)
print("Q4-->", Q4)
print("IQR-->", IQR)
print("Alt sınır: Q1 - 1.5 * IQR--->", Q1 - 1.5 * IQR)
print("Üst sınır: Q3 + 1.5 * IQR--->", Q3 + 1.5 * IQR)

In [ ]:
outliers_df = df[(df.Puan < (Q1–1.5*IQR))|(df.Puan > Q3+1.5*IQR)]